In [1]:
import xgboost as xgb
import pandas as pd
from datetime import date
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import TimeSeriesSplit
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

In [3]:
df = pd.read_csv("data/faturamento_consolidade_com_feriado_e_receita_da_categoria.csv", parse_dates=["datetime"])

In [16]:
submit_df = pd.read_csv("predicao.csv", parse_dates=["datetime"]).drop(["Unnamed: 0", "index"], axis=1)
submit_df = pd.get_dummies(submit_df, columns=["feature_05"])

str_cols = []
str_cols.append("datetime")
str_cols.append("receita")

str_cols.append("feature_01")
str_cols.append("feature_02")
str_cols.append("feature_11")
str_cols.append("feature_12")
str_cols.append("receita_trim1")
str_cols.append("receita_trim2")
str_cols.append("receita_trim3")
str_cols.append('receita_ano_passado') 

submit_df.head()

,cod_loja,datetime,cod_ap,cod_municipio,feature_01,feature_02,feature_03,feature_04,feature_06,feature_07,...,cnt_feriados,receita_minha_categoria,feature_05_FEATURE_05_VALUE_01,feature_05_FEATURE_05_VALUE_02,feature_05_FEATURE_05_VALUE_03,feature_05_FEATURE_05_VALUE_04,feature_05_FEATURE_05_VALUE_05,feature_05_FEATURE_05_VALUE_06,feature_05_FEATURE_05_VALUE_07,feature_05_FEATURE_05_VALUE_08
0,1,2018-01-01,4125506005002,4125506,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,1,21.76,31,0.168922,...,1.0,552910.464333,0,0,0,1,0,0,0,0
1,1,2018-02-01,4125506005002,4125506,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,1,21.76,31,0.168922,...,0.0,435209.493667,0,0,0,1,0,0,0,0
2,1,2018-03-01,4125506005002,4125506,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,1,21.76,31,0.168922,...,0.0,520654.621333,0,0,0,1,0,0,0,0
3,2,2018-01-01,5300108005004,5300108,FEATURE_01_VALUE_10,FEATURE_02_VALUE_03,1,22.00,27,0.215967,...,1.0,45986.626047,0,0,1,0,0,0,0,0
4,2,2018-02-01,5300108005004,5300108,FEATURE_01_VALUE_10,FEATURE_02_VALUE_03,1,22.00,27,0.215967,...,0.0,34970.681963,0,0,1,0,0,0,0,0


In [38]:
submit_laure_df = pd.read_csv("../eduardo/df_trimestral-submissao.csv").drop(["Unnamed: 0", "index"], axis=1)
variables = ['cod_ap', 'cod_municipio', 'feature_03', 'feature_04',
       'feature_06', 'feature_07', 'feature_08', 'feature_09', 'feature_10',
       'feature_13', 'feature_14', 'feature_15', 'feature_16', 'feature_17',
       'feature_18', 'quarter', 'ano', 'mes', 'categoria_loja',
       'receita_trim1', 'receita_trim2', 'receita_trim3', 'receita_media',
       'receita_ano_passado', 'cnt_feriados', 'receita_minha_categoria', 'MES',
       'loja', "index"]

submit_laure_df.head()

,indice,cod_ap,cod_municipio,feature_03,feature_04,feature_06,feature_07,feature_08,feature_09,feature_10,...,receita_trim1,receita_trim2,receita_trim3,receita_media,receita_ano_passado,cnt_feriados,receita_minha_categoria,MES,loja,data
0,0,4.125506e+12,4125506.0,1.0,21.76,31.0,0.168922,0.167659,115.958848,2.349821,...,637339.76,891173.22,1216996.84,915169.940000,611797.300,1.0,552910.464333,13.0,1.0,NaN
1,1,4.125506e+12,4125506.0,1.0,21.76,31.0,0.168922,0.167659,115.958848,2.349821,...,637339.76,891173.22,1216996.84,915169.940000,1067267.315,0.0,477932.057500,14.5,1.0,NaN
2,0,5.300108e+12,5300108.0,1.0,22.00,27.0,0.215967,0.197287,1562.674817,3.670883,...,59996.08,76670.53,92232.32,76299.643333,53209.150,1.0,45986.626047,13.0,2.0,2018-01-01
3,1,5.300108e+12,5300108.0,1.0,22.00,27.0,0.215967,0.197287,1562.674817,3.670883,...,59996.08,76670.53,92232.32,76299.643333,53830.140,0.0,34970.681963,14.0,2.0,2018-02-01
4,2,5.300108e+12,5300108.0,1.0,22.00,27.0,0.215967,0.197287,1562.674817,3.670883,...,59996.08,76670.53,92232.32,76299.643333,108589.210,0.0,41821.363387,15.0,2.0,2018-03-01


In [27]:
model_ikeda = CatBoostRegressor()
model_lauren = CatBoostRegressor()
model = CatBoostRegressor()

model_ikeda.load_model("modelo_catboost_otimizado_q234")
model_lauren.load_model("modelo_catboost_otimizado_lauren")
model.load_model("modelo_catboost_otimizado")

In [42]:
##model.predict(X_test)
#for _ in range(20):
pred_model = []
pred_ikeda = []
pred_mean = []
for cod_loja in df["cod_loja"].unique():
    #cod_loja = np.random.choice(df["cod_loja"].unique())
    print("Gerando para loja {}..".format(cod_loja))
    train_index = (df["cod_loja"] == cod_loja)
    test_index = (submit_df["cod_loja"] == cod_loja)
    
    #print(submit_laure_df[submit_laure_df["loja"] == cod_loja][variables].info())

    _train = df[train_index].sort_values("datetime")
    _test = submit_df[test_index].sort_values("datetime")
    
    col_to_drop = str_cols
    col_to_drop.append("cod_loja")
    y_pred_ikeda = model_ikeda.predict(submit_df[test_index].drop(col_to_drop, axis=1, errors="ignore"))
    y_pred = model.predict(submit_df[test_index].drop(col_to_drop, axis=1, errors="ignore"))
    y_pred_mean = (y_pred + y_pred_ikeda)/2
    
    pred_model.append([cod_loja, np.mean(y_pred)])
    pred_ikeda.append([cod_loja, np.mean(y_pred_ikeda)])
    pred_mean.append([cod_loja, np.mean(y_pred_mean)])

#     plt.figure(figsize=(9, 5))
#     plt.plot(_train["datetime"], _train["receita"], marker=".")
#     plt.plot(_test["datetime"], y_pred, label="Model", marker=".")
#     plt.plot(_test["datetime"], y_pred_ikeda, label="Ikeda", marker="x")
    
#     plt.plot(_test["datetime"], (y_pred + y_pred_ikeda)/2, label="Media", marker=".")
    
#     plt.legend()
#     plt.plot()
#     plt.show()

Gerando para loja 1..
Gerando para loja 2..
Gerando para loja 3..
Gerando para loja 4..
Gerando para loja 5..
Gerando para loja 6..
Gerando para loja 7..
Gerando para loja 8..
Gerando para loja 9..
Gerando para loja 11..
Gerando para loja 12..
Gerando para loja 14..
Gerando para loja 15..
Gerando para loja 17..
Gerando para loja 18..
Gerando para loja 19..
Gerando para loja 20..
Gerando para loja 22..
Gerando para loja 23..
Gerando para loja 24..
Gerando para loja 25..
Gerando para loja 26..
Gerando para loja 27..
Gerando para loja 29..
Gerando para loja 30..
Gerando para loja 31..
Gerando para loja 32..
Gerando para loja 33..
Gerando para loja 34..
Gerando para loja 35..
Gerando para loja 36..
Gerando para loja 37..
Gerando para loja 38..
Gerando para loja 39..
Gerando para loja 40..
Gerando para loja 42..
Gerando para loja 43..
Gerando para loja 44..
Gerando para loja 45..
Gerando para loja 46..
Gerando para loja 47..
Gerando para loja 48..
Gerando para loja 49..
Gerando para loja 50

Gerando para loja 416..
Gerando para loja 417..
Gerando para loja 418..
Gerando para loja 419..
Gerando para loja 421..
Gerando para loja 422..
Gerando para loja 423..
Gerando para loja 424..
Gerando para loja 425..
Gerando para loja 426..
Gerando para loja 428..
Gerando para loja 429..
Gerando para loja 430..
Gerando para loja 431..
Gerando para loja 432..
Gerando para loja 433..
Gerando para loja 434..
Gerando para loja 435..
Gerando para loja 436..
Gerando para loja 437..
Gerando para loja 438..
Gerando para loja 439..
Gerando para loja 440..
Gerando para loja 441..
Gerando para loja 442..
Gerando para loja 443..
Gerando para loja 444..
Gerando para loja 445..
Gerando para loja 446..
Gerando para loja 447..
Gerando para loja 449..
Gerando para loja 451..
Gerando para loja 453..
Gerando para loja 454..
Gerando para loja 455..
Gerando para loja 456..
Gerando para loja 457..
Gerando para loja 458..
Gerando para loja 459..
Gerando para loja 460..
Gerando para loja 462..
Gerando para loj

Gerando para loja 822..
Gerando para loja 823..
Gerando para loja 824..
Gerando para loja 825..
Gerando para loja 826..
Gerando para loja 827..
Gerando para loja 828..
Gerando para loja 829..
Gerando para loja 830..
Gerando para loja 831..
Gerando para loja 832..
Gerando para loja 833..
Gerando para loja 834..
Gerando para loja 835..
Gerando para loja 836..
Gerando para loja 837..
Gerando para loja 838..
Gerando para loja 839..
Gerando para loja 840..
Gerando para loja 842..
Gerando para loja 843..
Gerando para loja 844..
Gerando para loja 845..
Gerando para loja 846..
Gerando para loja 847..
Gerando para loja 848..
Gerando para loja 849..
Gerando para loja 850..
Gerando para loja 851..
Gerando para loja 852..
Gerando para loja 853..
Gerando para loja 854..
Gerando para loja 855..
Gerando para loja 856..
Gerando para loja 858..
Gerando para loja 859..
Gerando para loja 860..
Gerando para loja 862..
Gerando para loja 863..
Gerando para loja 864..
Gerando para loja 865..
Gerando para loj

Gerando para loja 1221..
Gerando para loja 1222..
Gerando para loja 1223..
Gerando para loja 1224..
Gerando para loja 1225..
Gerando para loja 1227..
Gerando para loja 1228..
Gerando para loja 1229..
Gerando para loja 1230..
Gerando para loja 1231..
Gerando para loja 1232..
Gerando para loja 1233..
Gerando para loja 1235..
Gerando para loja 1236..
Gerando para loja 1237..
Gerando para loja 1238..
Gerando para loja 1239..
Gerando para loja 1240..
Gerando para loja 1241..
Gerando para loja 1242..
Gerando para loja 1243..
Gerando para loja 1244..
Gerando para loja 1245..
Gerando para loja 1248..
Gerando para loja 1249..
Gerando para loja 1250..
Gerando para loja 1251..
Gerando para loja 1252..
Gerando para loja 1253..
Gerando para loja 1255..
Gerando para loja 1256..
Gerando para loja 1257..
Gerando para loja 1258..
Gerando para loja 1259..
Gerando para loja 1260..
Gerando para loja 1261..
Gerando para loja 1262..
Gerando para loja 1263..
Gerando para loja 1264..
Gerando para loja 1265..


Gerando para loja 1615..
Gerando para loja 1616..
Gerando para loja 1617..
Gerando para loja 1618..
Gerando para loja 1619..
Gerando para loja 1620..
Gerando para loja 1621..
Gerando para loja 1622..
Gerando para loja 1625..
Gerando para loja 1627..
Gerando para loja 1628..
Gerando para loja 1629..
Gerando para loja 1630..
Gerando para loja 1631..
Gerando para loja 1633..
Gerando para loja 1634..
Gerando para loja 1635..
Gerando para loja 1636..
Gerando para loja 1637..
Gerando para loja 1638..
Gerando para loja 1639..
Gerando para loja 1640..
Gerando para loja 1641..
Gerando para loja 1642..
Gerando para loja 1643..
Gerando para loja 1644..
Gerando para loja 1645..
Gerando para loja 1646..
Gerando para loja 1647..
Gerando para loja 1648..
Gerando para loja 1649..
Gerando para loja 1650..
Gerando para loja 1651..
Gerando para loja 1652..
Gerando para loja 1653..
Gerando para loja 1654..
Gerando para loja 1655..
Gerando para loja 1656..
Gerando para loja 1657..
Gerando para loja 1658..


Gerando para loja 2014..
Gerando para loja 2016..
Gerando para loja 2017..
Gerando para loja 2019..
Gerando para loja 2021..
Gerando para loja 2022..
Gerando para loja 2023..
Gerando para loja 2024..
Gerando para loja 2025..
Gerando para loja 2026..
Gerando para loja 2027..
Gerando para loja 2029..
Gerando para loja 2030..
Gerando para loja 2031..
Gerando para loja 2032..
Gerando para loja 2033..
Gerando para loja 2034..
Gerando para loja 2035..
Gerando para loja 2036..
Gerando para loja 2037..
Gerando para loja 2038..
Gerando para loja 2039..
Gerando para loja 2041..
Gerando para loja 2042..
Gerando para loja 2043..
Gerando para loja 2044..
Gerando para loja 2045..
Gerando para loja 2047..
Gerando para loja 2048..
Gerando para loja 2049..
Gerando para loja 2051..
Gerando para loja 2052..
Gerando para loja 2053..
Gerando para loja 2054..
Gerando para loja 2055..
Gerando para loja 2056..
Gerando para loja 2057..
Gerando para loja 2058..
Gerando para loja 2059..
Gerando para loja 2060..


Gerando para loja 2421..
Gerando para loja 2422..
Gerando para loja 2424..
Gerando para loja 2425..
Gerando para loja 2427..
Gerando para loja 2428..
Gerando para loja 2429..
Gerando para loja 2430..
Gerando para loja 2431..
Gerando para loja 2432..
Gerando para loja 2433..
Gerando para loja 2434..
Gerando para loja 2435..
Gerando para loja 2437..
Gerando para loja 2438..
Gerando para loja 2439..
Gerando para loja 2440..
Gerando para loja 2441..
Gerando para loja 2442..
Gerando para loja 2443..
Gerando para loja 2444..
Gerando para loja 2445..
Gerando para loja 2446..
Gerando para loja 2447..
Gerando para loja 2448..
Gerando para loja 2449..
Gerando para loja 2450..
Gerando para loja 2453..
Gerando para loja 2455..
Gerando para loja 2456..
Gerando para loja 2457..
Gerando para loja 2458..
Gerando para loja 2459..
Gerando para loja 2460..
Gerando para loja 2462..
Gerando para loja 2465..
Gerando para loja 2466..
Gerando para loja 2467..
Gerando para loja 2468..
Gerando para loja 2470..


Gerando para loja 2813..
Gerando para loja 2814..
Gerando para loja 2816..
Gerando para loja 2817..
Gerando para loja 2818..
Gerando para loja 2819..
Gerando para loja 2820..
Gerando para loja 2821..
Gerando para loja 2823..
Gerando para loja 2824..
Gerando para loja 2825..
Gerando para loja 2826..
Gerando para loja 2827..
Gerando para loja 2829..
Gerando para loja 2830..
Gerando para loja 2831..
Gerando para loja 2832..
Gerando para loja 2833..
Gerando para loja 2834..
Gerando para loja 2835..
Gerando para loja 2836..
Gerando para loja 2837..
Gerando para loja 2838..
Gerando para loja 2839..
Gerando para loja 2841..
Gerando para loja 2842..
Gerando para loja 2843..
Gerando para loja 2844..
Gerando para loja 2845..
Gerando para loja 2846..
Gerando para loja 2848..
Gerando para loja 2849..
Gerando para loja 2850..
Gerando para loja 2851..
Gerando para loja 2852..
Gerando para loja 2853..
Gerando para loja 2854..
Gerando para loja 2856..
Gerando para loja 2857..
Gerando para loja 2858..


Gerando para loja 3188..
Gerando para loja 3189..
Gerando para loja 3190..
Gerando para loja 3191..
Gerando para loja 3192..
Gerando para loja 3193..
Gerando para loja 3194..
Gerando para loja 3195..
Gerando para loja 3196..
Gerando para loja 3198..
Gerando para loja 3201..
Gerando para loja 3202..
Gerando para loja 3203..
Gerando para loja 3204..
Gerando para loja 3205..
Gerando para loja 3207..
Gerando para loja 3209..
Gerando para loja 3210..
Gerando para loja 3211..
Gerando para loja 3212..
Gerando para loja 3214..
Gerando para loja 3215..
Gerando para loja 3216..
Gerando para loja 3217..
Gerando para loja 3218..
Gerando para loja 3219..
Gerando para loja 3220..
Gerando para loja 3221..
Gerando para loja 3222..
Gerando para loja 3223..
Gerando para loja 3224..
Gerando para loja 3225..
Gerando para loja 3226..
Gerando para loja 3227..
Gerando para loja 3228..
Gerando para loja 3229..
Gerando para loja 3230..
Gerando para loja 3231..
Gerando para loja 3232..
Gerando para loja 3233..


Gerando para loja 3578..
Gerando para loja 3579..
Gerando para loja 3580..
Gerando para loja 3581..
Gerando para loja 3582..
Gerando para loja 3583..
Gerando para loja 3584..
Gerando para loja 3585..
Gerando para loja 3586..
Gerando para loja 3587..
Gerando para loja 3588..
Gerando para loja 3589..
Gerando para loja 3590..
Gerando para loja 3591..
Gerando para loja 3594..
Gerando para loja 3595..
Gerando para loja 3596..
Gerando para loja 3597..
Gerando para loja 3600..
Gerando para loja 3601..
Gerando para loja 3602..
Gerando para loja 3603..
Gerando para loja 3604..
Gerando para loja 3606..
Gerando para loja 3607..
Gerando para loja 3609..
Gerando para loja 3610..
Gerando para loja 3611..
Gerando para loja 3612..
Gerando para loja 3613..
Gerando para loja 3614..
Gerando para loja 3615..
Gerando para loja 3616..
Gerando para loja 3618..
Gerando para loja 3619..
Gerando para loja 3620..
Gerando para loja 3622..
Gerando para loja 3623..
Gerando para loja 3624..
Gerando para loja 3625..


In [51]:
pred_model_df = pd.DataFrame(pred_model, columns=["cod_loja", "faturamento"])
pred_ikeda_df = pd.DataFrame(pred_ikeda, columns=["cod_loja", "faturamento"])
pred_mean_df = pd.DataFrame(pred_mean, columns=["cod_loja", "faturamento"])

In [52]:
pred_model_df.to_csv("desafio_1_model.csv", index=False)
pred_ikeda_df.to_csv("desafio_1_ikeda.csv", index=False)
pred_mean_df.to_csv("desafio_1_mean.csv", index=False)